In [1]:
import itertools
import numpy as np
import torch
from torch import nn
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset

In [2]:
t = torch.tensor([[1, 2], [30, 40],[5,6],[70,80],[9,10]])
t.gather(0, torch.tensor([[0, 0],[2,2],[4,4],[1,1],[3,3]]))

tensor([[ 1,  2],
        [ 5,  6],
        [ 9, 10],
        [30, 40],
        [70, 80]])

In [3]:
from torch.nn.utils.rnn import pack_sequence
a = torch.tensor([1,2,3])
b = torch.tensor([4,5])
c = torch.tensor([6])
pack_sequence([c, a, b], enforce_sorted=False)

PackedSequence(data=tensor([1, 4, 6, 2, 5, 3]), batch_sizes=tensor([3, 2, 1]), sorted_indices=tensor([1, 2, 0]), unsorted_indices=tensor([2, 0, 1]))

In [4]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
print(z.grad_fn)

In [5]:
amino_acids = "arndcqeghilkmfpstwyv"
counter = itertools.count()
aa2index = {a: next(counter) for a in amino_acids}
dummy2index = {"a": 0, "b": 1}

def sequence_one_hot_encoder(indexer, sequence):
    dim = len(indexer)
    one_hot_encoded = np.zeros((len(sequence), dim+1))
    for i, aa in enumerate(sequence):
        index = indexer.get(aa, dim)
        one_hot_encoded[i, index] = 1.0
    return torch.tensor(one_hot_encoded, dtype=torch.float)

def pack_batch(batch):
    lengths = [x.size(0) for x in batch]   # get the length of each sequence in the batch\
    #print(f"sum of lengths {sum(lengths)}")
    #print(f"max of lengths {max(lengths)}")
    padded = nn.utils.rnn.pad_sequence(batch, batch_first=True)  # padd all sequences
    #b, s, n = padded.shape
    #print(f"padded shape {padded.shape}")
        
    # pack padded sequece
    packed = nn.utils.rnn.pack_padded_sequence(padded, lengths=lengths, batch_first=True, enforce_sorted=False)
    #packed = nn.utils.rnn.pack_sequence(batch, enforce_sorted=False)
        
    return packed, lengths

def batchify(batch):
    transposed_data = list(zip(*batch))
    batch1, batch2, labels = transposed_data
    
    return pack_batch(batch1), pack_batch(batch2), torch.tensor(labels)

class SequenceDataset(Dataset):
    def __init__(self, fpath, encoder):
        self.encoder = encoder
        with open(fpath, "r") as fin:
            self.lines = fin.readlines()

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        line = self.lines[idx]
        sline = line.strip().split("\t")
        return self.encoder(sline[0]), self.encoder(sline[1]), int(sline[4])
    
class DummyDataset(Dataset):
    def __init__(self, encoder):
        self.encoder = encoder
        self.lines = [
            ("ba", "bba","a.1.1.1","a.2.1.1",2),
            ("aba","aab","a.1.1.1","a.1.1.1",4),
        ]

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        sline = self.lines[idx]
        return self.encoder(sline[0]), self.encoder(sline[1]), int(sline[4])

In [6]:
dummy_data = DummyDataset(lambda x: sequence_one_hot_encoder(dummy2index, x))
#dummy_dataloader = DataLoader(dummy_data, batch_size=None, batch_sampler=None)
dummy_dataloader = DataLoader(dummy_data, batch_size=2, collate_fn=batchify)

In [37]:
class SequenceEmbedder(nn.Module):
    def __init__(self, input_dim, hidden_lstm_units=512, n_lstm_layers=1, output_dim=100, bidirectional=True):
        super(SequenceEmbedder, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_lstm_units = hidden_lstm_units
        self.n_lstm_layers = n_lstm_layers
        self.bidirectional = bidirectional
        self.output_dim = output_dim
        
        self.rnn = nn.RNN(
            input_size=hidden_lstm_units,
            hidden_size=hidden_lstm_units,
            num_layers=n_lstm_layers,
            batch_first=True,
            bidirectional=bidirectional,
            bias=False,
            nonlinearity="relu"
        ) 
        
        self.input_stack = nn.Sequential(
            nn.Linear(input_dim, hidden_lstm_units, dtype=torch.float),
            nn.ReLU()
        )
        
        self.output_stack = nn.Sequential(
            nn.Linear(hidden_lstm_units, output_dim, dtype=torch.float),
            nn.ReLU()
        )
        
    def _initialize_parameters(self):
        """
        Just to initialize a simple RNN with fixed values for debugging and understanding PyTorch forward flow
        Important note: will not work with LSTM or GRU, will not work with bias in RNN (or bidirectionality)
        """
        ones = np.ones((self.hidden_lstm_units, self.input_dim))
        for rowi in range(ones.shape[0]):
            for coli in range(ones.shape[1]):
                ones[rowi,coli] = rowi+coli
        self.input_stack[0].weight = Parameter(torch.tensor(
            ones,
            dtype=torch.float
        ))
        self.input_stack[0].bias = Parameter(torch.tensor(0, dtype=torch.float))
        
        for k in range(self.n_lstm_layers):
            ones = np.ones(getattr(self.rnn,f"weight_ih_l{k}").shape)
            setattr(self.rnn,f"weight_ih_l{k}",Parameter(torch.tensor(ones, dtype=torch.float)))
            ones = np.ones(getattr(self.rnn,f"weight_hh_l{k}").shape)
            setattr(self.rnn,f"weight_hh_l{k}",Parameter(torch.tensor(ones, dtype=torch.float)))

        ones = np.ones(self.output_stack[0].weight.shape)   
        self.output_stack[0].weight = Parameter(torch.tensor(
            ones,
            dtype=torch.float
        ))
        self.output_stack[0].bias = Parameter(torch.tensor(0, dtype=torch.float)) 
        
    def embed_sequence(self, batch, lengths):
        #unpack packed sequences in batch by padding: shape L*input_dim -> shape B*T*input_dim
        #where L is the sum of all sequence lenghts in the batch
        #where B is the batch size
        #where T is the length of the longest sequence in the batch
        batch_padded, _ = nn.utils.rnn.pad_packed_sequence(batch, batch_first=True)
        
        #relu transformation of input and packing sequences in batch for recurrent layer
        #shape B*T*input_dim -> L*hidden_lstm_units
        rnn_inputs = self.input_stack(batch_padded)
        rnn_inputs = nn.utils.rnn.pack_padded_sequence(rnn_inputs, lengths=lengths, batch_first=True, enforce_sorted=False)

        #apply recurrent layers and unpack by padding the output of the last recurrent layer
        rnn_output = self.rnn(rnn_inputs)
        rnn_out_unpacked, _ = nn.utils.rnn.pad_packed_sequence(rnn_output[0], batch_first=True)
        
        #feed output of recurrent layer to Relu unit
        return self.output_stack(rnn_out_unpacked)        
            
    def forward(self, batch):
        (batch1, lengths1), (batch2, lengths2), _ = batch
        
        sequence_embedding1 = self.embed_sequence(batch1, lengths1)
        sequence_embedding2 = self.embed_sequence(batch2, lengths2)
        
        return sequence_embedding1, sequence_embedding2
        

In [38]:
model = SequenceEmbedder(3, hidden_lstm_units=2, output_dim=2, bidirectional=False)
model._initialize_parameters()
model.float()

SequenceEmbedder(
  (rnn): RNN(2, 2, bias=False, batch_first=True)
  (input_stack): Sequential(
    (0): Linear(in_features=3, out_features=2, bias=True)
    (1): ReLU()
  )
  (output_stack): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU()
  )
)

In [39]:
for x in dummy_dataloader:
    out = model.forward(x)
    print(out[0],out[1])

tensor([[[ 6.,  6.],
         [14., 14.],
         [ 0.,  0.]],

        [[ 2.,  2.],
         [10., 10.],
         [22., 22.]]], grad_fn=<ReluBackward0>) tensor([[[ 6.,  6.],
         [18., 18.],
         [38., 38.]],

        [[ 2.,  2.],
         [ 6.,  6.],
         [18., 18.]]], grad_fn=<ReluBackward0>)


In [ ]:
train_data = SequenceDataset("../../data/train_set_0.tsv", lambda x: sequence_one_hot_encoder(aa2index, x))
train_dataloader = DataLoader(train_data, batch_size=None, batch_sampler=None)
#test_dataloader = DataLoader(test_data, batch_size=64)

64
torch.Size([89, 512])
torch.Size([11525, 512])
torch.Size([11525, 1024])
torch.Size([2, 64, 512])


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")